In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# Fix region names, remove underscore
def fix_region_name(roi):
    roi = roi.replace('_', ' ')
    roi = ' '.join(roi.split(',')[::-1]).strip()
    return roi

## Insert table path, figures path, weekly timepoints, last week

In [3]:
# table_path = Path('/data/schwartzao/covid-sicr/tables/20210317/')
# table_path = Path('/Users/schwartzao/Desktop/workspace/covid-sicr/tables/20210325/')
table_path = Path('/Users/schwartzao/Desktop/20210325/')



weeks = ['13', '30', '47']
weeks_dates = ['April 19-25, 2020', 'August 16-22, 2020', 'December 13-19, 2020'] # FOR COLUMN LABELS

last_week = 60 # use an int
last_week_str = 'March 14-20, 2021'

weeks_dates.append(last_week_str)


df = pd.read_csv(table_path / "fit_table_reweighted.csv") 

df['roi'] = df['roi'].apply(fix_region_name)

rois = list(df.roi.unique())

roi_us = np.sort([i for i in rois if i[:2]=='US'])
roi_other = np.sort([i for i in rois if i[:2]!='US'])

roi_other = list(roi_other) # removing super regions for now
roi_other = [x for x in roi_other if not x.startswith("AA")]

rois = list(roi_other) + list(roi_us)

stats = ['Rt', 'car', 'ifr', 'ir', 'ar']
for stat in stats:
    print(stat)
    df[f'{stat} {last_week_str}'] = 0
    for roi in rois:
        for wk in np.arange(last_week,0,-1):
    #         print(f"{stat} (week {wk})")
            if df.loc[df.roi==roi][f"{stat} (week {wk})"].notnull().values[0]:
                for q in df['quantile'].unique():
                    df.loc[(df.roi==roi)&(df['quantile']==q),[f'{stat} {last_week_str}']] = df.loc[(df.roi==roi)&(df['quantile']==q),[f"{stat} (week {wk})"]].values
    #                 print(df.loc[(df.roi==roi)&(df['quantile']==q)]['Rt (May 24th)'].values)
                break

Rt
car
ifr
ir
ar


In [4]:
theta = [stat + f' (week {week})' for stat in stats for week in weeks]
theta_last = [stat + f' {last_week_str}' for stat in stats]
theta_ = theta + theta_last

theta_R = [x for x in theta_ if x.startswith("R")]
theta_car = [x for x in theta_ if x.startswith("car")]
theta_ifr = [x for x in theta_ if x.startswith("ifr")]
theta_ir = [x for x in theta_ if x.startswith("ir")]
theta_ar = [x for x in theta_ if x.startswith("ar")]
print(theta_)

['Rt (week 13)', 'Rt (week 30)', 'Rt (week 47)', 'car (week 13)', 'car (week 30)', 'car (week 47)', 'ifr (week 13)', 'ifr (week 30)', 'ifr (week 47)', 'ir (week 13)', 'ir (week 30)', 'ir (week 47)', 'ar (week 13)', 'ar (week 30)', 'ar (week 47)', 'Rt March 14-20, 2021', 'car March 14-20, 2021', 'ifr March 14-20, 2021', 'ir March 14-20, 2021', 'ar March 14-20, 2021']


In [5]:
def afun1(x):
    return '%s' % float('%.1g' %x)

def afun2(x):
    return '%s' % float('%.2g' %x)

ir_col = [f"IRt ({weeks_dates[0]}) (CI)",
          f"IRt ({weeks_dates[1]}) (CI)", 
          f"IRt ({weeks_dates[2]}) (CI)",
          f"IRt ({weeks_dates[3]}) (CI)",
         ]

r_col = [f"R0 (CI)",
         f"Rt ({weeks_dates[0]}) (CI)", 
         f"Rt ({weeks_dates[1]}) (CI)",
         f"Rt ({weeks_dates[2]}) (CI)",
         f"Rt ({weeks_dates[3]}) (CI)"
        ]
car_col  = [f"CARt ({weeks_dates[0]}) (CI)", 
          f"CARt ({weeks_dates[1]}) (CI)",
          f"CARt ({weeks_dates[2]}) (CI)",
          f"CARt ({weeks_dates[3]}) (CI)"
         ]

ifr_col = [f"IFRt ({weeks_dates[0]}) (CI)", 
          f"IFRt ({weeks_dates[1]}) (CI)", 
          f"IFRt ({weeks_dates[2]}) (CI)",
          f"IFRt ({weeks_dates[3]}) (CI)"
         ]

ar_col = [f"ARt ({weeks_dates[0]}) (CI)",
          f"ARt ({weeks_dates[1]}) (CI)", 
          f"ARt ({weeks_dates[2]}) (CI)",
          f"ARt ({weeks_dates[3]}) (CI)"
         ]

stats = ['Rt', 'car', 'ifr', 'ir', 'ar']

for stat in stats:
    if stat == 'Rt':
        theta_stat = [x for x in theta_ if x.startswith("R")]
        theta_stat.insert(0, "R0")
        col = r_col

    if stat == 'car':
        theta_stat = [x for x in theta_ if x.startswith("car")]
        col = car_col
    if stat == 'ifr':
        theta_stat = [x for x in theta_ if x.startswith("ifr")]
        col = ifr_col

    if stat == 'ir':
        theta_stat = [x for x in theta_ if x.startswith("ir")]
        col = ir_col

    if stat == 'ar':
        theta_stat = [x for x in theta_ if x.startswith("ar")]
        col = ar_col
    
    rows = []
    for roi in rois:
        data = []
        data.append(roi)
        for i in range(len(theta_stat)):
            theta = theta_stat[i]

            mu = df.loc[(df.roi==roi)&(df['quantile']=='0.5'),theta].values[0]
            lb = df.loc[(df.roi==roi)&(df['quantile']=='0.025'),theta].values[0]
            ub = df.loc[(df.roi==roi)&(df['quantile']=='0.975'),theta].values[0]
            if theta[0] == 'R':
                x = afun2(mu)+" ("+afun2(lb)+", "+afun2(ub)+")"
            else:
                x = afun1(mu)+" ("+afun1(lb)+", "+afun1(ub)+")"
                
            data.append(x)
        rows.append(data)

        df_report = pd.DataFrame(rows, columns=['Region']+col)
        df_report.to_csv(table_path / f"{stat}_summary.csv", index=False)

## Report model contribution summaries

In [6]:
last_week = str(last_week)

df = pd.read_csv(table_path / 'fit_table_raw.csv', index_col=['model', 'roi', 'quantile'])
df = df[~df.index.duplicated(keep='last')]
df.columns.name = 'param'
df = df.stack('param').unstack(['roi', 'quantile', 'param']).T

last_week_stats = [x + f' (week {last_week})' for x in stats]
ll_waic_loo_aic = ['ll_', 'waic', 'loo', 'aic', 'num weeks', 'num_params']

rois = df.index.get_level_values('roi').unique()

dfs = []
for roi in rois:
    ll_waic_loo_aic_stats = df.loc[(roi, 'mean', ll_waic_loo_aic)]
    other_stats = df.loc[(roi, '0.5', last_week_stats)]
    dfs.append(ll_waic_loo_aic_stats)
    dfs.append(other_stats)
    
df_result = pd.concat(dfs)

# report minimum values for ll, waic, loo, aic
columns = [col for col in df if col.startswith('SICR')]
df_result = df_result.assign(minimum = df_result[columns].min(axis=1), minimum_column=df_result[columns].idxmin(axis=1))

df_result['outlier'] = ''
for roi in rois:
     # remove lowest value model values for non loo/waic/ll rows
    df_result.loc[(roi, 'mean', last_week_stats), 'minimum_column'] = ''
    df_result.loc[(roi, '0.5', last_week_stats), 'minimum_column'] = ''
    # find outliers
    if df_result.loc[(roi, '0.5', f'ir (week {last_week})'), "SICRdiscrete4Nwk"] > 0.9:
        if df_result.loc[(roi, 'mean', 'loo'), "minimum_column"] == 'SICRdiscrete4Nwk':
            print("possible outlier: ", roi)
            df_result.loc[(roi, 'mean', 'loo'), "outlier"] = "TRUE"
        
df_result.to_csv(table_path / 'model_contributions_and_median_stats.csv')

In [83]:
print(df_result)

model                           SICRdiscrete4Nwk        minimum  \
roi     quantile param                                            
Andorra mean     ll_                52448.983772   52448.983772   
                 loo                56787.730561   56787.730561   
                 waic              107276.981765  107276.981765   
                 num weeks             55.000000      55.000000   
                 num_params            26.000000      26.000000   
...                                          ...            ...   
US_CO   0.5      Rt (week 60)           0.485934       0.485934   
                 ar (week 60)      397923.992252  397923.992252   
                 car (week 60)          0.245720       0.245720   
                 ifr (week 60)          0.003253       0.003253   
                 ir (week 60)           0.594705       0.594705   

model                             minimum_column outlier  
roi     quantile param                                    
Andorra me